In [1]:
from dataset import get_es_loaders
from config import read_config
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using", device)

configs = read_config()
train_loader, val_loader, mix_train_set, mix_val_set, _train_set, _val_set = (
    get_es_loaders(configs, device)
)

Using cpu


In [2]:
from model import create_model

ckpt = torch.load("/kaggle/repo/emotion_intensity/rank_model_5.pt")
model, _kwargs = create_model("rank3")
model.load_state_dict(ckpt["state_dict"])


/kaggle/repo/emotion_intensity/model3.py:86: RuntimeWarning: divide by zero encountered in log
  torch.linspace(np.log(pitch_min), np.log(pitch_max), n_bins - 1)


<All keys matched successfully>

In [4]:
from torch.functional import F

def collate_fn(batchs):
    mels    = [torch.from_numpy(sample[0]) for sample in batchs]
    pitchs  = [torch.from_numpy(sample[1]) for sample in batchs]
    energys = [torch.from_numpy(sample[2]) for sample in batchs]
    emo_ids = [sample[3] for sample in batchs]
    basename = [sample[4] for sample in batchs]
    
    seq_lens = [len(mel) for mel in mels]
    max_len = max(seq_lens)
    
    mels = [F.pad(mel, (0, 0, 0, max_len - len(mel))) for mel in mels]
    pitchs = [F.pad(pitch, (0, max_len - len(pitch))) for pitch in pitchs]
    energys = [F.pad(energy, (0, max_len - len(energy))) for energy in energys]
    
    return torch.stack(mels), torch.stack(pitchs), torch.stack(energys), torch.tensor(emo_ids), seq_lens, basename

_val_set.list_files = list(filter(lambda x: x.replace(".pkl", "").split("_")[-1] != 'neutral', _val_set.list_files))

val_loader = torch.utils.data.DataLoader(
    _val_set, 
    batch_size=5, 
    shuffle=True,
    collate_fn=collate_fn
)

for idx, batch in enumerate(val_loader):
    mels, pitchs, energys, emo_ids, seq_lens, basename = batch
    i, h, r = model(mels, pitchs, energys, emo_ids)
    if idx == 1:
        break

In [34]:
F.sigmoid(r)

tensor([[0.9800],
        [0.0768],
        [0.0775],
        [0.9769],
        [0.0134]], grad_fn=<SigmoidBackward0>)

In [5]:
basename

['./datasets/esd_processed/mel/mel_0001_001472_surprise.pkl',
 './datasets/esd_processed/mel/mel_0011_000433_angry.pkl',
 './datasets/esd_processed/mel/mel_0001_000599_angry.pkl',
 './datasets/esd_processed/mel/mel_0001_000679_angry.pkl',
 './datasets/esd_processed/mel/mel_0011_000555_angry.pkl']

In [22]:
# play audio
from IPython.display import Audio
import IPython


for path, intent in zip(basename, r):
    print(path)
    print(intent)
    spk_id = path.split("_")[-3]
    sample_id = path.split("_")[-2]
    emo = path.replace(".pkl", "").split("_")[-1]
    #uppercase first letter
    emo = emo[0].upper() + emo[1:]
    IPython.display.display(Audio(
        f"./datasets/esd/{spk_id}/{emo}/{spk_id}_{sample_id}.wav", autoplay=True, rate=16000))


./datasets/esd_processed/mel/mel_0001_001472_surprise.pkl
tensor([-2.5073], grad_fn=<UnbindBackward0>)


./datasets/esd_processed/mel/mel_0011_000433_angry.pkl
tensor([-4.1435], grad_fn=<UnbindBackward0>)


./datasets/esd_processed/mel/mel_0001_000599_angry.pkl
tensor([-4.3478], grad_fn=<UnbindBackward0>)


./datasets/esd_processed/mel/mel_0001_000679_angry.pkl
tensor([-4.9513], grad_fn=<UnbindBackward0>)


./datasets/esd_processed/mel/mel_0011_000555_angry.pkl
tensor([-4.3369], grad_fn=<UnbindBackward0>)


In [18]:
!ls ./esd/0001/Surprise

ls: cannot access './esd/0001/Surprise': No such file or directory
